## ===================== Part 3 - LDA Topic Modeling =====================

## Preprocess Data & Build LDA Topics

#### Import packages

In [1]:
!pip install gensim
!pip install pyLDAvis
!pip install swifter
!pip install sentence_transformers


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 1.2/1.2 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 25.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ------------------ --------------------- 12.1/26.2 MB 62.7 MB/s eta 0:00:01
   ----------------------------------- ---- 23.6/26.2 MB 57.3 MB/s eta 0:00:01
   ---------------------------------------  26.2/26.2 MB 55.3 MB/s eta 0:00:01
   ---------------------------------------- 26.2/26.2 MB 37.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16514 sha256=974ccdbf654c0ae285e8c6348cf2e423c1f7c962a72da7497461530bc611d156
  Stored in directory: c:\users\user\appdata\local\pip\cache\whe

  DEPRECATION: Building 'swifter' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'swifter'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------------------------------ --------- 8.7/11.3 MB 41.3 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 29.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/558.8 kB ? eta -:--:--
   --------------------------------------- 558.8/558.8 kB 18.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   - -------------------------------------- 10.2/216.1 MB 48.8 MB/s eta 0:00:05
   --- ------------------------------------ 18.4/216.1 MB 44.4 MB/s eta 0:00:05
   ----- ---------------------------------- 27.8/216.1 MB 45.1 MB/s eta 0:00:05
   ------ --------------------------------- 37.5/216.1 MB 44.9 MB/s eta 0:00:04
   -------- ------------------------------- 48.2/216.1 MB 46.5 MB/s eta 0:00:04
   --


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore

from joblib import Parallel, delayed
import multiprocessing

import warnings
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# Silence warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### Setup & Downloads

In [4]:
# Setup & Downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

#### Load Dataset

In [8]:
import re
import json
from collections import defaultdict
from pymongo import MongoClient
import pandas as pd
from fractions import Fraction

import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unidecode

client = MongoClient("mongodb://localhost:27017/")
db = client["NLP_final"]
file = db["final_data"]

In [10]:
# make a copy of dataframe
df = file

In [11]:
df.columns

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NLP_final'), 'final_data.columns')

In [14]:
df = pd.DataFrame(list(file.find().limit(25000)))

#### convert cooking_time to minutes for all recipes

In [15]:
import re

def cooking_time_to_minutes(time_str):
    if pd.isna(time_str):
        return 0
    time_str = time_str.lower()
    hours = 0
    minutes = 0
    hr_match = re.search(r'(\d+)\s*hr', time_str)
    if hr_match:
        hours = int(hr_match.group(1))
    min_match = re.search(r'(\d+)\s*min', time_str)
    if min_match:
        minutes = int(min_match.group(1))
    return hours * 60 + minutes

# Convert cooking_time to minutes for all recipes once
df['cooking_time_mins'] = df['cooking_time'].apply(cooking_time_to_minutes)

#### Fill missing values and prepare full text field

In [16]:
# Fill missing values and prepare full text field
df['full_text'] = df['clean_ingredients'].fillna('') + ' ' + df['clean_directions'].fillna('')

#### Parallel Text Preprocessing / Tokenization

In [17]:
# Extend NLTK's stopwords with domain-specific cooking terms
custom_stopwords = set([
    "add", "cook", "heat", "bake", "boil", "fry", "stir", "mix", "remove", "place","pepper","bring","teaspoon","like","top",
    "pan", "pot", "oil", "water", "oven", "serve", "grill", "preheat", "use", "tsp", "salt","inch","two","cup","tbsp","bottom",
    "set", "let", "make", "prepare", "cut", "minutes", "cook", "temperature", "degrees","roll","bowl","one","tablespoon","turn",
    "take","get","hard", "side","put","surface","get","little","slow","dont", "mixture","medium","together","whole"
])

stop_words = set(stopwords.words('english')).union(custom_stopwords)
lemmatizer = WordNetLemmatizer()

In [18]:
# checking number of cores
num_cores = multiprocessing.cpu_count()
print(f"Using {num_cores} CPU cores for tokenization...")

Using 12 CPU cores for tokenization...


In [19]:
from joblib import Parallel, delayed
from tqdm import tqdm
import re
from nltk.tokenize import word_tokenize

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    tokens = word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(w)
        for w in tokens
        if w not in stop_words and len(w) > 2
    ]
    return tokens


# Wrap df['full_text'] with tqdm to show progress
tokens_list = Parallel(n_jobs=num_cores-1)(
    delayed(preprocess)(text) for text in tqdm(df['full_text'], desc="Preprocessing")
)

df['tokens'] = tokens_list

Preprocessing: 100%|███████████████████████████████████████████████████████████| 25000/25000 [00:20<00:00, 1238.90it/s]


In [20]:
df['tokens'].head()

0    [pound, flank, steak, finely, minced, green, o...
1    [rosemary, thyme, bay, leaf, smoked, paprika, ...
2    [carrot, butter, brown, sugar, grated, lemon, ...
3    [cup, flour, pinch, baking, powder, tbls, suga...
4    [crushed, small, thin, pretzel, stick, margari...
Name: tokens, dtype: object

#### Build Dictionary & Corpus for LDA

In [21]:
from tqdm import tqdm

# Create dictionary of tokens
dictionary = corpora.Dictionary(df['tokens'])

# Filter out extreme tokens
dictionary.filter_extremes(no_below=20, no_above=0.8, keep_n=10000)

# Create Bag-of-Words corpus
corpus = [dictionary.doc2bow(text) for text in tqdm(df['tokens'], desc="Creating BoW Corpus")]

Creating BoW Corpus: 100%|████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 28142.91it/s]


#### Train LDA Model (Parallel)

In [22]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


# Set number of topics
NUM_TOPICS = 10
print(f"Training LDA model with {NUM_TOPICS} topics using {num_cores-1} workers...")


# Train LDA model
lda_model = LdaMulticore(corpus=corpus,
                         id2word=dictionary,
                         num_topics=NUM_TOPICS,
                         workers=num_cores - 1,
                         passes=5,
                         chunksize=1000,  # default is 2000
                         random_state=42,
                         per_word_topics=True)

2025-08-05 23:50:42,750 : INFO : using symmetric alpha at 0.1
2025-08-05 23:50:42,751 : INFO : using symmetric eta at 0.1
2025-08-05 23:50:42,754 : INFO : using serial LDA version on this node
2025-08-05 23:50:42,766 : INFO : running online LDA training, 10 topics, 5 passes over the supplied corpus of 25000 documents, updating every 11000 documents, evaluating every ~25000 documents, iterating 50x with a convergence threshold of 0.001000
2025-08-05 23:50:42,768 : INFO : training LDA model using 11 processes


Training LDA model with 10 topics using 11 workers...


2025-08-05 23:51:03,872 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1000/25000, outstanding queue size 1
2025-08-05 23:51:03,881 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #2000/25000, outstanding queue size 2
2025-08-05 23:51:03,894 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #3000/25000, outstanding queue size 3
2025-08-05 23:51:03,905 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #4000/25000, outstanding queue size 4
2025-08-05 23:51:03,916 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to #5000/25000, outstanding queue size 5
2025-08-05 23:51:03,932 : INFO : PROGRESS: pass 0, dispatched chunk #5 = documents up to #6000/25000, outstanding queue size 6
2025-08-05 23:51:03,945 : INFO : PROGRESS: pass 0, dispatched chunk #6 = documents up to #7000/25000, outstanding queue size 7
2025-08-05 23:51:03,960 : INFO : PROGRESS: pass 0, dispatched chunk #7 = documents up to #8000/25000, outstandi

In [23]:
lda_model.print_topics()

2025-08-05 23:51:27,825 : INFO : topic #0 (0.100): 0.045*"cheese" + 0.020*"onion" + 0.018*"butter" + 0.017*"chopped" + 0.016*"egg" + 0.015*"bread" + 0.012*"cream" + 0.012*"milk" + 0.010*"soup" + 0.010*"large"
2025-08-05 23:51:27,826 : INFO : topic #1 (0.100): 0.032*"onion" + 0.025*"chopped" + 0.022*"tomato" + 0.019*"garlic" + 0.015*"sauce" + 0.014*"large" + 0.014*"chicken" + 0.013*"simmer" + 0.012*"rice" + 0.011*"cup"
2025-08-05 23:51:27,828 : INFO : topic #2 (0.100): 0.043*"cheese" + 0.026*"cream" + 0.016*"onion" + 0.014*"chopped" + 0.013*"sour" + 0.012*"ounce" + 0.012*"dough" + 0.011*"bean" + 0.011*"sauce" + 0.010*"shredded"
2025-08-05 23:51:27,829 : INFO : topic #3 (0.100): 0.018*"dough" + 0.015*"butter" + 0.012*"flour" + 0.011*"cool" + 0.010*"cup" + 0.009*"sugar" + 0.009*"sheet" + 0.009*"cake" + 0.009*"tablespoon" + 0.008*"large"
2025-08-05 23:51:27,830 : INFO : topic #4 (0.100): 0.045*"sugar" + 0.034*"egg" + 0.027*"flour" + 0.024*"butter" + 0.019*"vanilla" + 0.019*"baking" + 0.018

[(0,
  '0.045*"cheese" + 0.020*"onion" + 0.018*"butter" + 0.017*"chopped" + 0.016*"egg" + 0.015*"bread" + 0.012*"cream" + 0.012*"milk" + 0.010*"soup" + 0.010*"large"'),
 (1,
  '0.032*"onion" + 0.025*"chopped" + 0.022*"tomato" + 0.019*"garlic" + 0.015*"sauce" + 0.014*"large" + 0.014*"chicken" + 0.013*"simmer" + 0.012*"rice" + 0.011*"cup"'),
 (2,
  '0.043*"cheese" + 0.026*"cream" + 0.016*"onion" + 0.014*"chopped" + 0.013*"sour" + 0.012*"ounce" + 0.012*"dough" + 0.011*"bean" + 0.011*"sauce" + 0.010*"shredded"'),
 (3,
  '0.018*"dough" + 0.015*"butter" + 0.012*"flour" + 0.011*"cool" + 0.010*"cup" + 0.009*"sugar" + 0.009*"sheet" + 0.009*"cake" + 0.009*"tablespoon" + 0.008*"large"'),
 (4,
  '0.045*"sugar" + 0.034*"egg" + 0.027*"flour" + 0.024*"butter" + 0.019*"vanilla" + 0.019*"baking" + 0.018*"cream" + 0.016*"milk" + 0.012*"beat" + 0.012*"cup"'),
 (5,
  '0.026*"apple" + 0.026*"sugar" + 0.026*"juice" + 0.021*"lemon" + 0.015*"cup" + 0.014*"cinnamon" + 0.012*"tablespoon" + 0.011*"ounce" + 0.010

#### Show Top Words per Topic

In [24]:
# Get topic keywords for each topic
top_words_per_topic = []
for i in range(NUM_TOPICS):
    words = lda_model.show_topic(i, topn=10)
    topic_keywords = ", ".join([word for word, _ in words])
    top_words_per_topic.append((i, topic_keywords))

topic_df = pd.DataFrame(top_words_per_topic, columns=["Topic ID", "Top Keywords"])
display(topic_df)

,Topic ID,Top Keywords
0,0,"cheese, onion, butter, chopped, egg, bread, cr..."
1,1,"onion, chopped, tomato, garlic, sauce, large, ..."
2,2,"cheese, cream, onion, chopped, sour, ounce, do..."
3,3,"dough, butter, flour, cool, cup, sugar, sheet,..."
4,4,"sugar, egg, flour, butter, vanilla, baking, cr..."
5,5,"apple, sugar, juice, lemon, cup, cinnamon, tab..."
6,6,"sauce, onion, potato, meat, powder, garlic, br..."
7,7,"chicken, breast, cup, sugar, flour, rice, crea..."
8,8,"juice, cheese, chopped, pineapple, sugar, crea..."
9,9,"tablespoon, olive, garlic, sauce, fresh, large..."


#### Assign Topics to Each Recipe

In [25]:
# Assign dominant topic to each recipe
def get_dominant_topic(bow):
    topics = lda_model.get_document_topics(bow)
    if topics:
        return max(topics, key=lambda x: x[1])[0]
    return None

df['dominant_topic'] = [get_dominant_topic(doc) for doc in corpus]


#### Add topic name labels

In [26]:
# Add topic name labels for better UX
def get_topic_name(topic_id):
    return ", ".join([word for word, _ in lda_model.show_topic(topic_id, topn=3)])

df['topic_name'] = df['dominant_topic'].apply(get_topic_name)


#### Visualize with pyLDAvis

In [27]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.179391 -0.023983       1        1  20.138164
9     -0.107748  0.061212       2        1  15.573137
1     -0.135232 -0.009135       3        1  13.055232
3      0.134881  0.022550       4        1  12.012567
0     -0.041621 -0.086037       5        1  11.001237
6     -0.068264 -0.008733       6        1   6.779342
5      0.033323  0.117943       7        1   6.445344
8     -0.028387  0.030623       8        1   5.817177
2      0.005233 -0.101406       9        1   5.400717
7      0.028423 -0.003035      10        1   3.777082, topic_info=           Term          Freq         Total Category  logprob  loglift
50      chicken   9943.000000   9943.000000  Default  30.0000  30.0000
295      cheese  15293.000000  15293.000000  Default  29.0000  29.0000
36        sugar  22309.000000  22309.000000  Default  28.0000  28.0000
20        onion  16855.000000  16855.000000  Default  27.0000  27.0000
238       cream  13136.000000  13136.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
272       ounce    367.860105   9759.914558  Topic10  -5.0160  -0.0021
183       large    406.364898  14058.380626  Topic10  -4.9165  -0.2675
16   ingredient    325.942426   8329.327557  Topic10  -5.1370   0.0354
20        onion    346.426039  16855.985058  Topic10  -5.0761  -0.6086
131         egg    285.960607  15988.938797  Topic10  -5.2679  -0.7476

[755 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
237       1  0.063316  accent
237       2  0.031658  accent
237       3  0.015829  accent
237       5  0.110803  accent
237       6  0.712308  accent
...     ...       ...     ...
793       6  0.006293    zest
793       7  0.277665    zest
793       8  0.033823    zest
793       9  0.006293    zest
793      10  0.051128    zest

[4508 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 2, 4, 1, 7, 6, 9, 3, 8])

#### View dataframe

In [28]:
df.head()

,_id,Unnamed: 0,title,ingredients,directions,link,source,NER,clean_ingredients,clean_directions,clean_text,simhash,cooking_time,cuisine_tag,diet_tag,cooking_time_mins,full_text,tokens,dominant_topic,topic_name
0,688ef2b3a6ec3ca6894be311,2015528,Marinated Flank Steak Recipe,"[1 1/2 pound flank steak, 1/2 c. finely minced...","[Remove tenderloin from steak., Score meat., C...",cookeatshare.com/recipes/marinated-flank-steak...,Recipes1M,"[flank steak, green onions, red wine, soy sauc...",pound flank steak c finely minced green onions...,remove tenderloin from steak score meat combin...,pound flank steak c finely minced green onions...,2984177662168074987,6 hr 0 min,American,"Non-Vegetarian, Gluten-Free",360,pound flank steak c finely minced green onions...,"[pound, flank, steak, finely, minced, green, o...",9,"tablespoon, olive, garlic"
1,688ef2b3a6ec3ca6894be312,1608734,French Chicken Stew,"[1 tablespoon rosemary, 1 teaspoon thyme, 3 ba...",[combine all ingredients in slow cooker (6 qua...,www.yummly.com/recipe/French-Chicken-Stew-1433580,Gathered,"[rosemary, thyme, bay leaves, paprika, pepper,...",tablespoon rosemary teaspoon thyme bay leaves ...,combine all ingredients in slow cooker quarts ...,tablespoon rosemary teaspoon thyme bay leaves ...,16962350100183742163,6 hr 0 min,French,"Non-Vegetarian, Gluten-Free",360,tablespoon rosemary teaspoon thyme bay leaves ...,"[rosemary, thyme, bay, leaf, smoked, paprika, ...",1,"onion, chopped, tomato"
2,688ef2b3a6ec3ca6894be313,778500,Glazed Carrots,"[3 to 4 carrots, 1 1/2 Tbsp. butter, 1/3 c. br...",[Cook 3 to 4 carrots; cut crosswise in 1-inch ...,www.cookbooks.com/Recipe-Details.aspx?id=1011892,Gathered,"[carrots, butter, brown sugar, lemon rind]",to carrots tbsp butter c brown sugar grated le...,cook to carrots cut crosswise in inch pieces a...,to carrots tbsp butter c brown sugar grated le...,13631088686507211408,15 min,French,"Vegetarian, Gluten-Free",15,to carrots tbsp butter c brown sugar grated le...,"[carrot, butter, brown, sugar, grated, lemon, ...",5,"apple, sugar, juice"
3,688ef2b3a6ec3ca6894be314,1334975,Moms Pie Dough,"[4.5 Cups Flour, 1.5 Tsp Salt, Pinch Baking Po...","[Mix all dry ingredients in a bowl., , Add cri...",www.epicurious.com/recipes/member/views/moms-p...,Gathered,"[Flour, Salt, Baking Powder, Sugar, Crisco, eg...",cups flour tsp salt pinch baking powder tbls s...,mix all dry ingredients in a bowl add crisco a...,cups flour tsp salt pinch baking powder tbls s...,9758474554084259533,30 min,American,"Vegetarian, Contains Gluten",30,cups flour tsp salt pinch baking powder tbls s...,"[cup, flour, pinch, baking, powder, tbls, suga...",4,"sugar, egg, flour"
4,688ef2b3a6ec3ca6894be315,116562,Pretzel Salad Or Dessert,"[2 c. crushed small thin pretzels (sticks), 3/...","[Mix and press in baking pan, approximately 13...",www.cookbooks.com/Recipe-Details.aspx?id=106723,Gathered,"[thin pretzels, margarine]",c crushed small thin pretzels sticks c margarine,mix and press in baking pan approximately x in...,c crushed small thin pretzels sticks c margari...,12570825253960956135,8 min,German,"Vegan, Gluten-Free",8,c crushed small thin pretzels sticks c margari...,"[crushed, small, thin, pretzel, stick, margari...",2,"cheese, cream, onion"


### save dataset to drive

In [34]:
# Save DataFrame
df.to_csv('LDA_recipes.csv', index=False)

## ===================== Get User Preferences =====================

#### Get User Preferences via Input
- Accepts:
    - Diet tag
    - Cuisine tag
    - Max cooking time
    - Free-text query
    - (Optionally) ingredients

In [35]:
# Get User Preferences
def get_user_preferences():
    print("Enter diet preference (e.g., Vegetarian, Non-Vegetarian): ")
    diet = input().strip()

    print("Enter cuisine preference (e.g., Italian, French, Mexican): ")
    cuisine = input().strip()

    print("Enter max cooking time in minutes (e.g., 30): ")
    try:
        max_time = int(input().strip())
    except:
        max_time = 60  # fallback

    print("Enter your food idea / cravings (e.g., 'chocolate chip cookie', 'spicy tofu'): ")
    query_text = input().strip()

    print("Enter ingredients you want to include (comma-separated): ")
    ingredients = input().strip().split(',')

    return {
        "diet_tag": diet,
        "cuisine_tag": cuisine,
        "max_cooking_time_minutes": max_time,
        "query_text": query_text,
        "ingredients": [i.strip().lower() for i in ingredients if i.strip()]
    }

user_input = get_user_preferences()

Enter diet preference (e.g., Vegetarian, Non-Vegetarian): 


 Non-Vegetarian


Enter cuisine preference (e.g., Italian, French, Mexican): 


 Italain


Enter max cooking time in minutes (e.g., 30): 


 30


Enter your food idea / cravings (e.g., 'chocolate chip cookie', 'spicy tofu'): 


 spicy


Enter ingredients you want to include (comma-separated): 


## ===================== Part 4 - Recommendation System 1 =====================

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Load dataset
client = MongoClient("mongodb://localhost:27017/")
db = client["NLP_final"]
file = db["final_data"]

In [39]:
df2 = db["LDA"]

In [40]:
from sentence_transformers import SentenceTransformer, util
import torch
import swifter

## Sentence-BERT Embeddings

#### Format recipe to Generate Full Recipe Text (for Embedding)
- Combines key parts of each recipe into a single string used as input for Sentence-BERT.
- Format ensures the embedding captures meaningful semantic information.

In [41]:
# Format recipe to Generate Full Recipe Text (for Embedding)
def format_recipe(row):
    return f"Recipe: {row['title']}. Ingredients: {row['ingredients']}. Directions: {row['directions']}. Cooking time: {row['cooking_time']} minutes. Cuisine: {row['cuisine_tag']}. Diet: {row['diet_tag']}."

df['recipe_text'] = df.swifter.apply(format_recipe, axis=1)

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

#### Encode Recipes & User Query with Sentence-BERT Model
- Transforms each recipe's recipe_text into a dense vector (embedding).
- Converts the user's query into a vector.

In [42]:
# Loads lightweight sentence transformer model
# model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')  # Forces use of GPU
model = SentenceTransformer('all-MiniLM-L6-v2')

2025-08-06 00:02:02,197 : INFO : Use pytorch device_name: cpu
2025-08-06 00:02:02,198 : INFO : Load pretrained SentenceTransformer: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
import torch
print("GPU available:", torch.cuda.is_available())   # GPU must be avilable to run faster

GPU available: False


In [44]:
# Encodes each recipe's text into a vector (embedding) that captures its meaning  with Sentence-BERT
texts = df['recipe_text'].tolist()

recipe_embeddings = model.encode(
    texts,
    batch_size=256,
    show_progress_bar=True,
    convert_to_tensor=True
)

Batches:   0%|          | 0/98 [00:00<?, ?it/s]

In [45]:
import torch

# Save recipe embeddings
torch.save(recipe_embeddings, 'recipe_embeddings.pt')
df.to_csv('processed_recipes.csv', index=False)

In [46]:
# Load recipe embeddings
recipe_embeddings = torch.load('recipe_embeddings.pt')
df = pd.read_csv('processed_recipes.csv')



# Encode User Query with Sentence-BERT
user_embedding = model.encode(user_input["query_text"], convert_to_tensor=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### Computes cosine similarity between the user query and each recipe.

In [47]:
# Computes cosine similarity between the user query and each recipe
cos_sim = util.cos_sim(user_embedding, recipe_embeddings).cpu().numpy().flatten()

#### Calculate Ingredient Overlap
- Measures overlap between user-specified ingredients and recipe’s ingredients.

In [48]:
# Calculate Ingredient Overlap
def ingredient_overlap(row_ingredients):
    recipe_ings = set(str(row_ingredients).lower().split(', '))
    user_ings = set(user_input["ingredients"])
    return len(recipe_ings & user_ings) / max(len(recipe_ings | user_ings), 1)

df['ingredient_overlap'] = df['clean_ingredients'].apply(ingredient_overlap)

#### Calculate Tag Match (diet & cuisine)
- Returns 1.0 if both match, 0.5 if one matches, 0 if none.

In [49]:
# Calculate Tag Match (diet & cuisine)
df['tag_match'] = (
    (df['diet_tag'].str.lower() == user_input['diet_tag'].lower()).astype(int) +
    (df['cuisine_tag'].str.lower() == user_input['cuisine_tag'].lower()).astype(int)
) / 2

#### Calculate Topic Match
- Checks if the recipe and user query share the same dominant topic.

In [50]:
# Calculate Topic Match
query_tokens = preprocess(user_input['query_text'])
query_bow = dictionary.doc2bow(query_tokens)
query_topic_dist = lda_model.get_document_topics(query_bow)
query_dominant_topic = max(query_topic_dist, key=lambda x: x[1])[0]
df['topic_match'] = (df['dominant_topic'] == query_dominant_topic).astype(int)

#### Filter Recipes by Cooking Time

In [51]:
# Filter by cooking time
df_filtered = df[df['cooking_time_mins'] <= user_input['max_cooking_time_minutes']].copy()

# Normalize cooking time so that faster recipes score higher
max_time = user_input['max_cooking_time_minutes']
df_filtered['cooking_time_score'] = 1 - (df_filtered['cooking_time_mins'] / max_time)
df_filtered['cooking_time_score'] = df_filtered['cooking_time_score'].clip(0, 1)

####  Calculate Final Score

In [52]:
# Final Score
df_filtered['score'] = (
    0.5 * cos_sim[df_filtered.index] +
    0.2 * df_filtered['ingredient_overlap'] +
    0.2 * df_filtered['tag_match'] +
    0.1 * df_filtered['topic_match'] +
    0.1  * df_filtered['cooking_time_score']
)

## Generate Recommendations

#### Return Top N Recommendations

In [53]:
# Top 5 Recommendations
top_5 = df_filtered.sort_values('score', ascending=False).head(5)


print("\n🎯 Top 5 Recommended Recipes Based on Your Preferences:\n")
for i, row in enumerate(top_5.itertuples(), 1):
    print(f"{i}. {row.title} "
          f"(Score: {row.score:.3f}, "
          f"Cooking Time: {row.cooking_time_mins} mins, "
          f"Cuisine: {row.cuisine_tag}, "
          f"Diet: {row.diet_tag})")



🎯 Top 5 Recommended Recipes Based on Your Preferences:

1. Spicy Barbecue Sauce (Score: 0.368, Cooking Time: 5 mins, Cuisine: American, Diet: Vegan, Gluten-Free)
2. The Best Hot Sauce Recipe (Score: 0.353, Cooking Time: 5 mins, Cuisine: Mexican, Diet: Vegan, Gluten-Free)
3. Russian Salad (Ensalada Russa) (Score: 0.348, Cooking Time: 5 mins, Cuisine: French, Diet: Vegan, Gluten-Free)
4. Doc'S Barbecue Sauce (Score: 0.346, Cooking Time: 1 mins, Cuisine: Chinese, Diet: Vegetarian, Contains Gluten)
5. Spicy Fried Tofu (Score: 0.346, Cooking Time: 5 mins, Cuisine: Italian, Diet: Vegetarian, Gluten-Free)


## Shopping list

#### MongoDB connecting (depends on where we store the ingredient dataset, saved in the share file)

In [55]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["NLP_final"]
recipes = db["final_data"]  # where recipe content is stored
shopping_list = db["sl"]    # where cleaned ingredients are stored


Load top 5 recommended titles from df_filtered

In [56]:
top_5 = df_filtered.sort_values('score', ascending=False).head(5)
recommended_titles = top_5["title"].tolist()

Load ingredient unit mappings from sl collection (which is where i save my ingredient file)

In [57]:
from collections import defaultdict
from fractions import Fraction
import re

# Build cleaned ingredients and units from sl collection
cleaned_ingredients = set()
ingredient_units = {}

for item in shopping_list.find({}, {"ingredient": 1, "unit": 1}):
    name = item.get("ingredient")
    if not isinstance(name, str):
        continue
    name = name.strip().lower()
    unit = (item.get("unit") or "unit").strip().lower()
    cleaned_ingredients.add(name)
    ingredient_units[name] = unit


Normalization and Fuzzy Matching

In [58]:
def match_ingredient(name: str) -> str:
    name = name.lower()
    for clean in cleaned_ingredients:
        if clean in name or name in clean:
            return clean
    return name

def normalize_ingredient(raw: str):
    raw = raw.replace(",", "").replace(".", "").strip()
    pattern = r"""^\s*
        (\d+\s\d+/\d+|\d+/\d+|\d+)?       # quantity
        \s*
        (cup|cups|tbsp|tablespoon|tsp|teaspoon|oz|ounce|pound|lb|
         clove|can|slice|handful|stalk|medium|large|small|unit|g|kg|ml|l)?
        \s+
        (.+)$                             # ingredient name
    """
    match = re.match(pattern, raw, re.IGNORECASE | re.VERBOSE)

    if not match:
        return raw.lower(), "unit", 1.0

    qty_str, unit, name = match.groups()
    name = name.strip().lower()
    unit = (unit or "unit").lower()

    try:
        qty = float(sum(Fraction(part) for part in qty_str.split())) if qty_str else 1.0
    except Exception:
        qty = 1.0

    matched_name = match_ingredient(name)
    final_unit = ingredient_units.get(matched_name, unit)

    return matched_name, final_unit, qty


#### Aggregate ingredients from top 5 recipes

In [68]:
def aggregate_ingredients(titles):
    totals = defaultdict(lambda: {
        "quantity": 0.0,
        "unit": "",
        "raw": [],
        "recipes": set()
    })

    for title in titles:
        recipe = recipes.find_one({"title": title}, {"ingredients": 1})
        if not recipe:
            print(f"Recipe not found: {title}")
            continue

        for raw in recipe.get("ingredients", []):
            name, unit, qty = normalize_ingredient(raw)
            totals[name]["quantity"] += qty
            totals[name]["unit"] = unit
            totals[name]["raw"].append(raw)
            totals[name]["recipes"].add(title)

    return totals


Run the full connection

In [65]:
result = aggregate_ingredients(recommended_titles)

print("\n🛒 Aggregated Shopping List from Top 5 Recommended Recipes:\n")
for name, info in result.items():
    print(f"- {name}: {info['quantity']} {info['unit']} (Raw: {info['raw']})")



🛒 Aggregated Shopping List from Top 5 Recommended Recipes:

- c: 6.166666666666666 cup (Raw: ['1 c. ketchup', '2/3 c. butter (1 1/3 sticks)', '1/2 to 3/4 tsp. curry', '2 cloves garlic, pressed, or 1 tsp. garlic powder', '2 to 3 Tbsp. soy sauce'])
- heinz ketchup balsamic vinegar: 1.1666666666666665 cup (Raw: ['1/2 c. vinegar', '2/3 c. vinegar'])
- chipotle chiles ground chili powder: 2.0 teaspoon (Raw: ['2 Tbsp. chili powder'])
- dry mustard ground black pepper: 3.0 teaspoon (Raw: ['2 Tbsp. dry mustard', '1 tablespoon mustard'])
- either light dark brown sugar: 6.0 cup (Raw: ['6 Tbsp. brown sugar'])
- salt pepper worcestershire sauce: 8.0 unit (Raw: ['6 Tbsp. Worcestershire sauce', '2 tsp. Worcestershire sauce'])
- margarine sunflower oil: 1.0 teaspoon (Raw: ['1 tsp. margarine'])
- ice: 5.0 unit (Raw: ['4 (28 ounce) cans diced tomatoes with juice', '1 large onion, sliced into rings'])
- bunches cilantro divided: 2.0 unit (Raw: ['2 bunches cilantro'])
- white: 0.125 ounce (Raw: ['1/8 c

#### for a better look

In [69]:
# Convert result dictionary to a DataFrame for better formatting
df_shopping = pd.DataFrame([
    {
        "Ingredient": name,
        "Total Quantity": round(info["quantity"], 2),
        "Unit": info["unit"],
        "Raw Sources": ", ".join(info["raw"]),
        "Used In": ", ".join(sorted(info["recipes"]))
    }
    for name, info in result.items()
])

# Sort alphabetically by ingredient name
df_shopping = df_shopping.sort_values("Ingredient").reset_index(drop=True)

# Display the DataFrame
from IPython.display import display
display(df_shopping)


,Ingredient,Total Quantity,Unit,Raw Sources,Used In
0,basil powdered,0.50,teaspoon,"1/2 tsp. each: turmeric, sweet basil, powdered...",Spicy Fried Tofu
1,beans drain,1.00,unit,"1 (48 oz.) can green beans, drained",Doc'S Barbecue Sauce
2,beef stock canned unsalted broth,2.00,cup,"1 tsp. salt, 1 tsp. salt",Doc'S Barbecue Sauce
3,bunches cilantro divided,2.00,unit,2 bunches cilantro,The Best Hot Sauce Recipe
4,c,6.17,cup,"1 c. ketchup, 2/3 c. butter (1 1/3 sticks), 1/...","Doc'S Barbecue Sauce, Spicy Barbecue Sauce, Sp..."
5,c parmesan,0.25,unit,1/4 to 1/3 c. Parmesan cheese or nutritional y...,Spicy Fried Tofu
6,c sugar plus muffin tin,0.75,cup,3/4 c. sugar,Doc'S Barbecue Sauce
7,c water broth,0.67,unit,2/3 c. water,Doc'S Barbecue Sauce
8,chipotle chiles ground chili powder,2.00,teaspoon,2 Tbsp. chili powder,Spicy Barbecue Sauce
9,dill weed fine,1.00,teaspoon,1 tsp. dill weed,Spicy Fried Tofu


In [2]:
import subprocess

notebook_filename = "LDA_&_Reccomendation_shoppinglist_intergrated.ipynb"

# Use the "classic" template instead of "lab"
subprocess.run([
    "jupyter", "nbconvert", "--to", "html", "--template", "classic", notebook_filename
])


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'html', '--template', 'classic', 'LDA_&_Reccomendation_shoppinglist_intergrated.ipynb'], returncode=1)